In [51]:
import pandas as pd
import sqlite3 as sql
import json
import os
from pprint import pprint

In [2]:
path_to_cord = '../../../../../media/arnav/Arnav/ArnavCode/projects/cord-19'

In [11]:
df = pd.read_csv(path_to_cord + '/metadata.csv')

/home/arnav/anaconda3/envs/deep-learning/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [12]:
df.drop(['sha', 'source_x', 'pmcid', 'mag_id', 'who_covidence_id', 'arxiv_id', 'pmc_json_files', 's2_id'], 
        axis=1, 
        inplace=True)

In [42]:
def prepare_sliced_df(start, end) :
    df_slice = df[start:end]
    df_slice = df_slice.dropna()
    
    #get pdf_json_file location
    pdf_json_file = df_slice.iloc[:, 9]
    
    text = []
    for i, location in enumerate(pdf_json_file) :
        
        path = path_to_cord + '/' + str(location)
        
        if os.path.isfile(path) :

            f = open(path)

            # returns JSON object as a dictionary
            data = json.load(f)

            text_temp = ''

            count_text = 0
            for body in data['body_text'] :
                text_temp += body['text']
                count_text += 1
                if count_text > 3 :
                    break

            text.append(text_temp)

            # Closing file
            f.close()
            
        else : text.append('')
            
    df_slice['body_text'] = text
    return df_slice

In [43]:
df_slice = prepare_sliced_df(0, 5000)


In [46]:
df_slice.head()

,cord_uid,title,doi,pubmed_id,license,abstract,publish_time,authors,journal,pdf_json_files,url,body_text
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,document_parses/pdf_json/d1aafb70c066a2068b027...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Mycoplasma pneumoniae is a common cause of upp...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,document_parses/pdf_json/6b0567729c2143a66d737...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Since its discovery as a biological messenger ...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,10.1186/rr19,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,document_parses/pdf_json/06ced00a5fc04215949aa...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Surfactant protein-D (SP-D) is a member of the...
3,2b73a28n,Role of endothelin-1 in lung disease,10.1186/rr44,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,document_parses/pdf_json/348055649b6b8cf2b9a37...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,from Xenopus laevis [16] . ETA receptors in no...
4,9785vg6d,Gene expression in epithelial cells in respons...,10.1186/rr61,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,document_parses/pdf_json/5f48792a5fa08bed9f560...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,RSV and PVM are viruses of the family Paramyxo...


In [49]:
conn = sql.connect('cord.db')
df_slice.to_sql('cord19', conn, if_exists='replace', index=False)

In [50]:
pd.read_sql('select * from cord19 limit 5', conn)

,cord_uid,title,doi,pubmed_id,license,abstract,publish_time,authors,journal,pdf_json_files,url,body_text
0,ug7v899j,Clinical features of culture-proven Mycoplasma...,10.1186/1471-2334-1-6,11472636,no-cc,OBJECTIVE: This retrospective chart review des...,2001-07-04,"Madani, Tariq A; Al-Ghamdi, Aisha A",BMC Infect Dis,document_parses/pdf_json/d1aafb70c066a2068b027...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3...,Mycoplasma pneumoniae is a common cause of upp...
1,02tnwd4m,Nitric oxide: a pro-inflammatory mediator in l...,10.1186/rr14,11667967,no-cc,Inflammatory diseases of the respiratory tract...,2000-08-15,"Vliet, Albert van der; Eiserich, Jason P; Cros...",Respir Res,document_parses/pdf_json/6b0567729c2143a66d737...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Since its discovery as a biological messenger ...
2,ejv2xln0,Surfactant protein-D and pulmonary host defense,10.1186/rr19,11667972,no-cc,Surfactant protein-D (SP-D) participates in th...,2000-08-25,"Crouch, Erika C",Respir Res,document_parses/pdf_json/06ced00a5fc04215949aa...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,Surfactant protein-D (SP-D) is a member of the...
3,2b73a28n,Role of endothelin-1 in lung disease,10.1186/rr44,11686871,no-cc,Endothelin-1 (ET-1) is a 21 amino acid peptide...,2001-02-22,"Fagan, Karen A; McMurtry, Ivan F; Rodman, David M",Respir Res,document_parses/pdf_json/348055649b6b8cf2b9a37...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,from Xenopus laevis [16] . ETA receptors in no...
4,9785vg6d,Gene expression in epithelial cells in respons...,10.1186/rr61,11686888,no-cc,Respiratory syncytial virus (RSV) and pneumoni...,2001-05-11,"Domachowske, Joseph B; Bonville, Cynthia A; Ro...",Respir Res,document_parses/pdf_json/5f48792a5fa08bed9f560...,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5...,RSV and PVM are viruses of the family Paramyxo...
